# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [51]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = (22,12)
plt.rcParams['font.size'] = 14

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')#.set_index('CustomerID') 
data.head(3)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [18]:
ratings = data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index()
ratings.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [39]:
len(ratings.CustomerID.unique())

1000

In [40]:
len(ratings.ProductName.unique())

452

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [4]:
ratings_pivot = pd.pivot_table(ratings, columns='CustomerID', index='ProductName', values='Quantity').fillna(0)
ratings_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [5]:
dist = squareform(pdist(ratings_pivot.T,'euclidean')) #Cálculo de la distancia entre usuarios vía la distancia euclidiana.
afinidad = 1/(1+dist)                                 #Definición de ajuste de valores: mientras mayor sea el número, mayor similitud.
afinidad = pd.DataFrame(afinidad, index=ratings_pivot.columns, columns=ratings_pivot.columns)     #Convirtiéndolo en un df.
afinidad

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
464,0.082709,0.075255,0.080350,0.078187,0.078711,1.000000,0.074798,0.084626,0.080070,0.078187,...,0.004816,0.004687,0.004416,0.005038,0.004308,0.004533,0.004578,0.004363,0.004167,0.004347
477,0.074573,0.075956,0.076923,0.075025,0.075025,0.074798,1.000000,0.078977,0.075720,0.074573,...,0.004823,0.004688,0.004426,0.005037,0.004321,0.004534,0.004589,0.004366,0.004168,0.004338
639,0.083020,0.076435,0.080634,0.082403,0.082403,0.084626,0.078977,1.000000,0.085983,0.083020,...,0.004816,0.004682,0.004412,0.005035,0.004308,0.004526,0.004581,0.004363,0.004176,0.004335
649,0.081503,0.077674,0.082100,0.077171,0.078187,0.080070,0.075720,0.085983,1.000000,0.079792,...,0.004810,0.004676,0.004414,0.005035,0.004314,0.004528,0.004585,0.004369,0.004172,0.004329


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [6]:
random_id = data.CustomerID.sample(1).iloc[0]
top_most_similar = afinidad[random_id].sort_values(ascending=False).index.tolist()[1:6]
top_most_similar

[12561, 20901, 15888, 28702, 10311]

In [7]:
#Los top 5 más similares al customer con id #15796

smf15796 = afinidad[15796].sort_values(ascending=False)[1:6]
smf15796

CustomerID
3544    0.027778
3253    0.027305
3885    0.027212
3074    0.027181
669     0.027098
Name: 15796, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
top_most_similar_df = data[data.CustomerID.isin(top_most_similar)]
top_most_similar_df.head(10)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
350,10311,Blake,Wilson,3351021,153,Arizona - Green Tea,3
727,20901,Donnell,Woodard,3005092,428,Wine - Ruffino Chianti,6
1357,15888,Jermaine,Reid,3296516,159,Eggplant - Asian,5
1515,28702,Christy,Walters,2699325,196,Longos - Grilled Salmon With Bbq,8
1518,15888,Jermaine,Reid,2812740,196,Longos - Grilled Salmon With Bbq,5
1536,20901,Donnell,Woodard,746534,196,Longos - Grilled Salmon With Bbq,6
1669,20901,Donnell,Woodard,5716946,69,Raspberries - Fresh,6
1670,20901,Donnell,Woodard,6157545,69,Raspberries - Fresh,6
1793,28702,Christy,Walters,853959,43,Sponge Cake Mix - Chocolate,8
1922,10311,Blake,Wilson,929630,244,Beef - Rib Eye Aaa,3


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
#the 5 most similar customers that I selected: top_most_similar = [15796, 31113, 27821, 14208, 11102]

total_number_prod = top_most_similar_df.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
total_number_prod.head(12)

ProductName
Banana Turning                      29
Scallops - Live In Shell            28
Lime Cordial - Roses                26
Carbonated Water - Cherry           21
Bread - Multigrain                  21
Water - Aquafina Vitamin            21
Snapple - Iced Tea Peach            20
Longos - Grilled Salmon With Bbq    19
Otomegusa Dashi Konbu               18
Soupfoamcont12oz 112con             17
Wasabi Powder                       17
Brandy - Bar                        17
Name: Quantity, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
recommendation = [product for product in total_number_prod.index if not product in data['ProductName'][data.CustomerID == random_id].unique()]

total_number_prod[total_number_prod.index.isin(recommendation)][:5]       

ProductName
Longos - Grilled Salmon With Bbq    19
Otomegusa Dashi Konbu               18
Soupfoamcont12oz 112con             17
Brandy - Bar                        17
Fuji Apples                         17
Name: Quantity, dtype: int64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
top_most_similar = afinidad[random_id].sort_values(ascending=False)[1:6].index.tolist()
top_most_similar

[12561, 20901, 15888, 28702, 10311]

In [30]:
# Recapitulando:

# Filtro los datos como los necesito, haciendo la matriz (cuadrada) por producto y customer id
#ratings_pivot = pd.pivot_table(data, index='ProductName', columns='CustomerID', values ='Quantity').fillna(0)

#Hago df con el cálculo de la similitud 
#dist = squareform(pdist(ratings_pivot.T,'euclidean'))
#afinidad = 1/(1+dist)                                
#afinidad = pd.DataFrame(afinidad, index=ratings_pivot.columns, columns=ratings_pivot.columns)

#Itero desde el paso 4 hasta el 7 con los customers top 5

recommendation_dict ={}
for ID in data.CustomerID.unique():
    
    #Selecciono los top 5 clientes más similares (del paso 4)
    top_most_similar = afinidad[ID].sort_values(ascending=False)[1:6].index.tolist()
    
    #Filtro  por Product Quantity df de los top 5 customers más afines (del paso 5)
    top_most_similar_df = ratings[ratings.CustomerID.isin(top_most_similar)]
    
    # Ordeno los 5 productos por customers afines (del paso 6)
    total_number_prod = top_most_similar_df.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
    
    #Obtengo la lista de los productos recomendados por customer (del paso 7)
    recommendation = [product for product in total_number_prod.index if not product in data['ProductName'][data.CustomerID == random_id].unique()]
    #Añado el customer id con los 5 productos top recomendados
    recommendation_dict[ID] = total_number_prod[total_number_prod.index.isin(recommendation)][:5].index.tolist()

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [31]:
euclidean_recommendation = pd.DataFrame(recommendation_dict).T
euclidean_recommendation.columns = ['1st recommendation','2nd recommendation','3rd recommendation','4th recommendation','5th recommendation']
euclidean_recommendation.head()

,1st recommendation,2nd recommendation,3rd recommendation,4th recommendation,5th recommendation
61288,Pork - Kidney,"Cheese - Boursin, Garlic / Herbs",Ice Cream Bar - Hageen Daz To,Bay Leaf,Wine - Ej Gallo Sierra Valley
77352,Quiche Assorted,Veal - Sweetbread,"Cheese - Brie, Triple Creme",Tia Maria,Pasta - Angel Hair
40094,Butter - Unsalted,"Pepper - Black, Whole",Tia Maria,Sugar - Fine,Soupfoamcont12oz 112con
23548,Pork - Kidney,Beef - Texas Style Burger,"Juice - Cranberry, 341 Ml",Veal - Sweetbread,Lamb - Ground
78981,Watercress,Veal - Sweetbread,Knife Plastic - White,Hersey Shakes,"Pork - Bacon, Double Smoked"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [28]:
# Filtro los datos como los necesito, haciendo la matriz (cuadrada) por producto y customer id
ratings_pivot = pd.pivot_table(data, index='ProductName', columns='CustomerID', values ='Quantity').fillna(0)

#Hago df con el cálculo de la similitud 
dist = squareform(pdist(ratings_pivot.T,'cityblock'))
afinidad = 1/(1+dist)                                
afinidad = pd.DataFrame(afinidad, index=ratings_pivot.columns, columns=ratings_pivot.columns)

#Itero desde el paso 4 hasta el 7 con los customers top 5
recommendation_dict ={}
for ID in data.CustomerID.unique():
    
    #Selecciono los top 5 clientes más similares (del paso 4)
    top_most_similar = afinidad[ID].sort_values(ascending=False)[1:6].index.tolist()
    
    #Filtro  por Product Quantity df de los top 5 customers más afines (del paso 5)
    top_most_similar_df = ratings[ratings.CustomerID.isin(top_most_similar)]
    
    # Ordeno los 5 productos por customers afines (del paso 6)
    total_number_prod = top_most_similar_df.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
    
    #Obtengo la lista de los productos recomendados por customer (del paso 7)
    recommendation = [product for product in total_number_prod.index if not product in data['ProductName'][data.CustomerID == random_id].unique()]
    #Añado el customer id con los 5 productos top recomendados
    recommendation_dict[ID] = total_number_prod[total_number_prod.index.isin(recommendation)][:5].index.tolist()

In [29]:
cityblock_recommendation = pd.DataFrame(recommendation_dict).T
cityblock_recommendation.columns = ['1st recommendation','2nd recommendation','3rd recommendation','4th recommendation','5th recommendation']
cityblock_recommendation.head()

,1st recommendation,2nd recommendation,3rd recommendation,4th recommendation,5th recommendation
61288,Pork - Kidney,"Cheese - Boursin, Garlic / Herbs",Ice Cream Bar - Hageen Daz To,Bay Leaf,Wine - Ej Gallo Sierra Valley
77352,Quiche Assorted,Veal - Sweetbread,"Cheese - Brie, Triple Creme",Tia Maria,Pasta - Angel Hair
40094,Butter - Unsalted,"Pepper - Black, Whole",Tia Maria,Sugar - Fine,Soupfoamcont12oz 112con
23548,Pork - Kidney,Beef - Texas Style Burger,"Juice - Cranberry, 341 Ml",Veal - Sweetbread,Lamb - Ground
78981,Watercress,Veal - Sweetbread,Knife Plastic - White,Hersey Shakes,"Pork - Bacon, Double Smoked"
